In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import warnings
from pandas.tseries.offsets import BDay
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import scipy.optimize as sco

warnings.filterwarnings("ignore")

## 데이터 불러오기

In [14]:
import os
print(os.listdir('drive/MyDrive/dongkuk'))

['mvp_code_250113.ipynb', 'parsed_result_sector_prompt_version_31_0106_cot_10y_all.csv', 'parsed_result_sector_prompt_version_31_1104_10y_all.csv']


In [17]:
df = pd.read_csv('drive/MyDrive/dongkuk/parsed_result_sector_prompt_version_31_0106_cot_10y_all.csv')
tbl = pd.read_csv('drive/MyDrive/dongkuk/parsed_result_sector_prompt_version_31_1104_10y_all.csv')

In [18]:
#df = df.merge(pt[['permno', 'year', 'month', 'ticker']], on=['permno', 'year', 'month'], how='inner')
df = df.merge(tbl[['permno','ticker', 'year', 'month', 'shrout']], on=['permno', 'year', 'month'], how='inner')
df['market_cap'] = df['shrout'] * df['prc']
df['date'] = pd.to_datetime(df['date'])

## yfinance: Adjusted Close 불러오기

In [19]:
# 실패한 티커 제거
failed_tickers = [
    'BBT', 'CERN', 'ABC', 'TMK', 'UTX', 'JEC', 'VAR', 'SYMC', 'COG', 'CBS', 'WRK', 'BHGE', 'ARNC', 'HRS', 'PBCT',
    'BLL', 'PKI', 'XLNX', 'VIAC', 'CTL', 'NLOK', 'DISCA', 'FLIR', 'GPS', 'PEAK', 'PXD', 'ANTM', 'CTXS', 'CBG',
    'PCLN', 'COH', 'BF', 'MWV', 'KFT', 'WLP', 'LTD', 'TYC', 'HCN', 'ZMH', 'MHFI', 'LB', 'FISV'
]

# 실패한 티커 제거
unique_tickers = df['ticker'].unique().tolist()
unique_tickers = [ticker for ticker in unique_tickers if ticker not in failed_tickers]

In [20]:
# 주가 데이터 다운로드
stock_data = pd.DataFrame()
for ticker_name in unique_tickers:
      ticker = yf.Ticker(ticker_name)
      all_prices = ticker.history(start='2012-01-01', end='2021-12-31', auto_adjust=True)['Close']
      all_prices = all_prices.dropna()
      #all_prices.index = all_prices.index.tz_localize(None)
      stock_data[ticker_name] = all_prices

In [21]:
stock_data

,ORCL,MSFT,TROW,ADM,KO,ED,XRAY,FAST,DTE,ETN,...,AMP,ICE,MA,WU,ROST,DFS,TEL,PM,V,UNH
Date,,,,,,,,,,,,,,,,,,,,,
2012-01-03 00:00:00-05:00,21.361288,21.120092,37.084251,20.325317,23.424976,37.023548,31.363714,15.593018,29.230970,32.527004,...,38.460148,20.531527,34.181133,10.535896,20.826664,18.656424,24.586605,42.004974,23.458218,42.204330
2012-01-04 00:00:00-05:00,21.485182,21.617134,37.013420,20.663019,23.278028,36.565708,31.354843,15.806810,29.084457,32.519844,...,37.981750,20.263130,33.056530,10.393903,20.971987,18.733479,24.656895,41.930149,23.039116,42.819077
2012-01-05 00:00:00-05:00,21.964291,21.838032,37.534996,20.571558,23.167818,36.468040,31.381441,16.081200,29.296087,32.648552,...,38.303173,20.261402,32.713959,10.445022,22.205122,18.895311,24.930256,41.801884,23.212233,43.105980
2012-01-06 00:00:00-05:00,22.245140,22.177280,37.805454,20.451952,23.020870,36.077351,31.195290,16.109695,29.041056,32.383984,...,38.191044,19.915081,31.833012,10.399583,22.346056,18.725779,25.273907,41.197926,22.938900,43.261700
2012-01-09 00:00:00-05:00,22.377604,21.885378,37.477051,20.339380,23.020870,36.181129,31.044582,16.095440,29.041056,32.562756,...,38.198536,19.494305,32.102230,10.433663,22.187511,18.802843,25.625374,41.080330,22.754400,43.212532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-23 00:00:00-05:00,85.610207,326.229187,171.536865,60.581974,53.192986,75.444992,52.344223,57.887344,105.680023,159.514603,...,288.469666,130.269257,353.735168,14.276908,106.368561,107.546402,151.114594,79.506920,211.681839,474.433350
2021-12-27 00:00:00-05:00,85.438454,333.793030,175.076706,61.165195,53.585850,75.408981,52.835266,58.802963,106.611801,162.011185,...,293.346252,132.100906,353.470276,14.196702,108.502510,108.273705,155.253662,80.028824,212.668854,478.379181
2021-12-28 00:00:00-05:00,84.589218,332.623383,175.155792,61.961361,53.795986,76.012383,52.910824,59.080425,107.407898,162.514282,...,291.704803,131.985809,353.931366,14.268891,108.299721,108.227074,155.062927,80.302582,213.079254,481.673706


## **Buy & Hold**
  1. Buy the stock on the first day.

  2. Hold the stock until the last day.

In [22]:
# 전체 기간 동안 Buy & Hold 수익률 계산
buy_hold_returns = []
for ticker in unique_tickers:
      start_price = stock_data[ticker].iloc[0]
      end_price = stock_data[ticker].iloc[-1]
      ret = (end_price - start_price) / start_price
      buy_hold_returns.append(ret)

buy_hold_df = pd.DataFrame({
    'ticker': unique_tickers,
    'return': buy_hold_returns
}).dropna()

rf = yf.download('^IRX', start='2012-01-01', end='2021-12-31', auto_adjust=True, progress=False)['Close'].mean().iloc[0] / 100

mean_return_buy_hold = buy_hold_df['return'].mean() # 평균 수익률
std_return_buy_hold = buy_hold_df['return'].std() # 연 수익률의 표준편차
sharpe_ratio_buy_hold = (mean_return_buy_hold - rf) / std_return_buy_hold # Sharpe Ratio

print(">>> Buy & Hold (10년)")
print("Returns:", mean_return_buy_hold)
print("Standard Deviation of Returns:", std_return_buy_hold)
print("Sharpe Ratio:", sharpe_ratio_buy_hold)

>>> Buy & Hold (10년)
Returns: 4.161142833856162
Standard Deviation of Returns: 6.7139218641938445
Sharpe Ratio: 0.6189222811800222


In [23]:
buy_hold_df

,ticker,return
0,ORCL,2.931343
1,MSFT,14.660072
2,TROW,3.693104
3,ADM,2.045294
4,KO,1.292623
...,...,...
304,DFS,4.789068
305,TEL,5.229226
306,PM,0.919887
307,V,8.075853


## **논문**
- 매년 6/30에 리밸런싱
- decimal portfolio(각각 long, short portfolio가 전체의 10%가 되도록)
    - long: 모델이 증가한다고 예측한 주식에 대해, logprob 내림차순 정렬 후 전체의 10%가 되도록 추출
    - short: 모델이 감소한다고 예측한 주식에 대해, logprob 내림차순 정렬 후 전체의 10%가 되도록 추출

In [24]:
# 연도별로 6월 마지막 거래일 찾기
def get_last_trading_day(stock_data, year, month=6):
    june_trading_days = stock_data.loc[(stock_data.index.month == month) & (stock_data.index.year == year)].index
    return june_trading_days[-1] if not june_trading_days.empty else None

# 롱/숏 포트폴리오 구성
long_portfolio = []
short_portfolio = []

# 각 연도의 6월 마지막 거래일에 포트폴리오를 구성하고, 예측 값에 따라 포트폴리오 구성
for year in df['year'].unique():
    portfolio_date = get_last_trading_day(stock_data, year) # 해당 연도의 6월 마지막 거래일 찾기
    current_data = df[df['year'] == year] # 해당 연도의 모든 데이터 필터링

    unique_tickers = current_data['ticker'].unique() # 연도별 전체 주식 수 계산
    top_10_percent_count = max(1, int(len(unique_tickers) * 0.1))  # 고유한 티커의 10%, 최소 1개 보장

    # long 포트폴리오
    long_stocks = current_data[
        (current_data['return_movement'] > 0) &
        (current_data['magnitude_of_change'].isin(['Moderate', 'Large']))
    ]
    long_top_by_ticker = long_stocks.loc[long_stocks.groupby('ticker')['logprobs'].idxmax()]  # 티커별 logprobs 최대값 선택
    long_selected = long_top_by_ticker.sort_values(by='logprobs', ascending=False).head(top_10_percent_count)
    long_selected['portfolio_date'] = portfolio_date
    long_portfolio.append(long_selected)

    # short 포트폴리오
    short_stocks = current_data[
        (current_data['return_movement'] == 0) &
        (current_data['magnitude_of_change'].isin(['Moderate', 'Large']))
    ]
    short_top_by_ticker = short_stocks.loc[short_stocks.groupby('ticker')['logprobs'].idxmax()]  # 티커별 logprobs 최대값 선택
    short_selected = short_top_by_ticker.sort_values(by='logprobs', ascending=False).head(top_10_percent_count)
    short_selected['portfolio_date'] = portfolio_date
    short_portfolio.append(short_selected)

long_portfolio_df = pd.concat(long_portfolio, ignore_index=True)
short_portfolio_df = pd.concat(short_portfolio, ignore_index=True)

# 각 연도의 6월 마지막 거래일부터 다음 해 6월 마지막 거래일까지 보유
long_portfolio_df['hold_start'] = long_portfolio_df['year'].apply(lambda x: get_last_trading_day(stock_data, x))
long_portfolio_df['hold_end'] = long_portfolio_df['year'].apply(lambda x: get_last_trading_day(stock_data, x + 1))
long_portfolio_df['hold_period'] = long_portfolio_df['hold_start'].dt.year.astype(str) + '-' + long_portfolio_df['hold_end'].dt.year.astype(str)

short_portfolio_df['hold_start'] = short_portfolio_df['year'].apply(lambda x: get_last_trading_day(stock_data, x))
short_portfolio_df['hold_end'] = short_portfolio_df['year'].apply(lambda x: get_last_trading_day(stock_data, x + 1))
short_portfolio_df['hold_period'] = short_portfolio_df['hold_start'].dt.year.astype(str) + '-' + short_portfolio_df['hold_end'].dt.year.astype(str)

In [25]:
long_portfolio_df

,date,permno,prc,mom1m,mom12m,chmom,indmom,maxret,mom36m,turn,...,logprobs,token_log_prob,avg_logprob,ticker,shrout,market_cap,portfolio_date,hold_start,hold_end,hold_period
0,2012-03-30,48506,42.10000,0.041365,0.181242,0.288362,-0.088736,0.027763,0.836983,1.610713,...,-0.165111,"**: -0.008398452773690224, Prediction: -0.0317...",-0.165111,MCO,223315.0,9.401562e+06,2012-06-29 00:00:00-04:00,2012-06-29 00:00:00-04:00,2013-06-28 00:00:00-04:00,2012-2013.0
1,2012-03-30,52476,44.26000,0.083397,0.104841,0.398873,-0.088736,0.076396,0.691815,1.231183,...,-0.166744,"**: -0.007004112936556339, Prediction: -0.0221...",-0.166744,EFX,120182.0,5.319255e+06,2012-06-29 00:00:00-04:00,2012-06-29 00:00:00-04:00,2013-06-28 00:00:00-04:00,2012-2013.0
2,2012-12-31,27043,70.24000,0.035950,0.072795,0.236365,0.100018,0.030108,0.331408,1.892002,...,-0.168250,"**: -0.005205410998314619, Prediction: -0.0284...",-0.168250,VAR,109175.0,7.668452e+06,2012-06-29 00:00:00-04:00,2012-06-29 00:00:00-04:00,2013-06-28 00:00:00-04:00,2012-2013.0
3,2012-12-31,19561,75.36000,0.060761,0.044414,0.054238,0.134781,0.031981,0.377968,1.312011,...,-0.168955,"**: -0.005899161100387573, Prediction: -0.0221...",-0.168955,BA,754077.0,5.682724e+07,2012-06-29 00:00:00-04:00,2012-06-29 00:00:00-04:00,2013-06-28 00:00:00-04:00,2012-2013.0
4,2012-07-31,34746,13.82000,0.009738,0.068039,0.053911,0.050861,0.035568,0.821240,2.767210,...,-0.170243,"**: -0.005677763372659683, Prediction: -0.0173...",-0.170243,FITB,918913.0,1.269938e+07,2012-06-29 00:00:00-04:00,2012-06-29 00:00:00-04:00,2013-06-28 00:00:00-04:00,2012-2013.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,2021-06-30,77730,73.76000,0.032214,0.286073,0.158627,0.445647,0.017686,-0.048632,1.468294,...,-0.173450,"**: -0.0057559930719435215, Prediction: -0.015...",-0.173450,TSN,294773.0,2.174246e+07,2021-06-30 00:00:00-04:00,2021-06-30 00:00:00-04:00,NaT,2021-nan
316,2021-03-31,10107,235.77000,0.004225,0.442759,-0.363548,0.780086,0.033152,0.779927,0.767173,...,-0.173557,"**: -0.006511304993182421, Prediction: -0.0175...",-0.173557,MSFT,7534000.0,1.776291e+09,2021-06-30 00:00:00-04:00,2021-06-30 00:00:00-04:00,NaT,2021-nan
317,2021-04-30,60943,49.67000,0.064248,0.671161,-0.447194,2.564368,0.071041,-0.348831,1.619814,...,-0.173764,"**: -0.0065874564461410046, Prediction: -0.026...",-0.173764,LEG,133155.0,6.613809e+06,2021-06-30 00:00:00-04:00,2021-06-30 00:00:00-04:00,NaT,2021-nan
318,2021-05-28,19561,247.02000,-0.080127,0.806269,0.598860,1.407077,0.025300,-0.558182,5.834525,...,-0.173832,"**: -0.005445646122097969, Prediction: -0.0184...",-0.173832,BA,584810.0,1.444598e+08,2021-06-30 00:00:00-04:00,2021-06-30 00:00:00-04:00,NaT,2021-nan


In [27]:
long_portfolio_df['return']

,return
0,0.691981
1,0.283611
2,NaN
3,0.411716
4,0.383270
...,...
315,NaN
316,NaN
317,NaN
318,NaN


### Equal-weighted Portfolio

In [26]:
# 포트폴리오 수익률 계산
def calculate_portfolio_returns(portfolio_df, stock_data, is_short=False):
    returns = []
    for _, row in portfolio_df.iterrows():
        ticker = row['ticker']
        try:
            start_price = stock_data.loc[row['hold_start'], ticker]
            end_price = stock_data.loc[row['hold_end'], ticker]
            ret = (start_price - end_price) / start_price if is_short else (end_price - start_price) / start_price
            returns.append(ret)
        except KeyError:
            returns.append(np.nan)
    return returns

# 롱/숏 포트폴리오 수익률 계산
long_portfolio_df['return'] = calculate_portfolio_returns(long_portfolio_df, stock_data)
short_portfolio_df['return'] = calculate_portfolio_returns(short_portfolio_df, stock_data, is_short=True)

In [28]:
# 'hold_period' 기준으로 Equal-weighted 가중치 계산
long_weights = long_portfolio_df.groupby('hold_period')['ticker'].nunique().apply(lambda x: 1 / x)
short_weights = short_portfolio_df.groupby('hold_period')['ticker'].nunique().apply(lambda x: 1 / x)

long_portfolio_df['weight'] = long_portfolio_df['hold_period'].map(long_weights)
short_portfolio_df['weight'] = short_portfolio_df['hold_period'].map(short_weights)

# 가중치 적용 수익률 계산
long_portfolio_df['weighted_return'] = long_portfolio_df['return'] * long_portfolio_df['weight']
short_portfolio_df['weighted_return'] = short_portfolio_df['return'] * short_portfolio_df['weight']

# 연도별 수익률의 평균
long_portfolio_return = long_portfolio_df.groupby('hold_period')['weighted_return'].sum().mean()
short_portfolio_return = short_portfolio_df.groupby('hold_period')['weighted_return'].sum().mean()
long_short_return = long_portfolio_return - short_portfolio_return

# 연도별 표준 편차의 평균
long_portfolio_std = np.sqrt(long_portfolio_df.groupby('hold_period').apply(
    lambda x: np.sum((x['return'] - long_portfolio_return) ** 2 * x['weight'])
).mean())
short_portfolio_std = np.sqrt(short_portfolio_df.groupby('hold_period').apply(
    lambda x: np.sum((x['return'] - short_portfolio_return) ** 2 * x['weight'])
).mean())
long_short_std = np.sqrt(long_portfolio_std**2 + short_portfolio_std**2)

# 연도별 sharpe ratio
sharpe_long = (long_portfolio_return - rf) / long_portfolio_std
sharpe_short = (short_portfolio_return - rf) / short_portfolio_std
sharpe_long_short = (long_short_return - rf) / long_short_std

# 결과값 출력
results_equal_weighted = pd.DataFrame({
    "Portfolio": ["Long Portfolio", "Short Portfolio", "Long-Short Portfolio"],
    "Ret": [round(long_portfolio_return, 4), round(short_portfolio_return, 4), round(long_short_return, 4)],
    "Std": [round(long_portfolio_std, 4), round(short_portfolio_std, 4), round(long_short_std, 4)],
    "Sharpe Ratio": [round(sharpe_long, 4), round(sharpe_short, 4), round(sharpe_long_short, 4)]
})

print(">>> Equal_weighted portfolio")
results_equal_weighted

>>> Equal_weighted portfolio


,Portfolio,Ret,Std,Sharpe Ratio
0,Long Portfolio,0.1532,0.2381,0.6195
1,Short Portfolio,-0.1363,0.2502,-0.5675
2,Long-Short Portfolio,0.2895,0.3454,0.8215


### Value-weighted portfolio

In [29]:
# 'hold_period' 기준으로 Value-weighted 가중치 계산
long_portfolio_df['weight'] = long_portfolio_df.groupby('hold_period')['market_cap'].transform(lambda x: x / x.sum())
short_portfolio_df['weight'] = short_portfolio_df.groupby('hold_period')['market_cap'].transform(lambda x: x / x.sum())

# 가중치 적용 수익률 계산
long_portfolio_df['weighted_return'] = long_portfolio_df['return'] * long_portfolio_df['weight']
short_portfolio_df['weighted_return'] = short_portfolio_df['return'] * short_portfolio_df['weight']

# 연도별 수익률의 평균
long_portfolio_return = long_portfolio_df.groupby('hold_period')['weighted_return'].sum().mean()
short_portfolio_return = short_portfolio_df.groupby('hold_period')['weighted_return'].sum().mean()
long_short_return = long_portfolio_return - short_portfolio_return

# 연도별 표준 편차의 평균
long_portfolio_std = np.sqrt(long_portfolio_df.groupby('hold_period').apply(
    lambda x: np.sum((x['return'] - long_portfolio_return) ** 2 * x['weight'])
).mean())
short_portfolio_std = np.sqrt(short_portfolio_df.groupby('hold_period').apply(
    lambda x: np.sum((x['return'] - short_portfolio_return) ** 2 * x['weight'])
).mean())
long_short_std = np.sqrt(long_portfolio_std**2 + short_portfolio_std**2)

# 연도별 sharpe ratio
sharpe_long = (long_portfolio_return - rf) / long_portfolio_std
sharpe_short = (short_portfolio_return - rf) / short_portfolio_std
sharpe_long_short = (long_short_return - rf) / long_short_std

# 결과값 출력
results_value_weighted = pd.DataFrame({
    "Portfolio": ["Long Portfolio", "Short Portfolio", "Long-Short Portfolio"],
    "Ret (평균 수익률)": [round(long_portfolio_return, 4), round(short_portfolio_return, 4), round(long_short_return, 4)],
    "Std (표준 편차)": [round(long_portfolio_std, 4), round(short_portfolio_std, 4), round(long_short_std, 4)],
    "Sharpe Ratio": [round(sharpe_long, 4), round(sharpe_short, 4), round(sharpe_long_short, 4)]
})
print(">>> Value_weighted portfolio")
results_value_weighted

>>> Value_weighted portfolio


,Portfolio,Ret (평균 수익률),Std (표준 편차),Sharpe Ratio
0,Long Portfolio,0.1643,0.2190,0.7238
1,Short Portfolio,-0.1531,0.2405,-0.6606
2,Long-Short Portfolio,0.3174,0.3253,0.9580


## Mean-variance Framework

### Maximum Sharpe Ratio Portfolio
- 샤프지수를 최대로 하는 포트폴리오
- The tangency portfolio is derived by maximizing the Sharpe Ratio (SR) (equivalent to solving Eq. (4) with 𝝁 ≠ 0), which gauges the degree of diversification achieved at the portfolio level

### Minumum risk portfolio
- The minimum risk portfolio is derived by minimizing the variance of the portfolio (equivalent to solving Eq. (4) with 𝝁 = 0), which measures the level of risk.

### Maxiumum return portfolio

In [30]:
# 일간 수익률
daily_returns = stock_data.pct_change().dropna()

# 연평균 수익률
annual_returns = daily_returns.mean() * 252  # 연간 거래일 수:252

# 자산의 분산-공분산 행렬
cov_matrix = daily_returns.cov() * 252

mu = annual_returns.values  # 자산의 기대 수익률 벡터
cov_matrix = cov_matrix.values  # 자산의 분산-공분산 행렬

In [31]:
def max_sharpe_ratio(w, mu, cov_matrix, rf):
    portfolio_return = np.dot(w, mu)
    portfolio_volatility = np.sqrt(np.dot(w.T, np.dot(cov_matrix, w)))
    return -(portfolio_return - rf) / portfolio_volatility  # 음수로 변환하여 minimize 사용

def min_risk(w, cov_matrix):
    return np.sqrt(np.dot(w.T, np.dot(cov_matrix, w)))

def max_return(w, mu):
    return -np.dot(w, mu)  # 음수로 변환하여 minimize 사용

def calculate_portfolio_metrics(weights, mu, cov_matrix, rf):
    portfolio_return = np.dot(weights, mu) # return
    portfolio_std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) # std
    sharpe_ratio = (portfolio_return - rf) / portfolio_std # sharpe ratio
    return portfolio_return, portfolio_std, sharpe_ratio

In [32]:
# 연도별 결과 저장 딕셔너리
results = {
    "Maximum Sharpe Ratio": {"return": [], "std": [], "sharpe": []},
    "Minimum Risk": {"return": [], "std": [], "sharpe": []},
    "Maximum Return": {"return": [], "std": [], "sharpe": []}
}

In [33]:
# 연도별로 6월 마지막 거래일 추출
rebalancing_dates = [get_last_trading_day(stock_data, year) for year in range(2012, 2022)]
for i in range(len(rebalancing_dates) - 1):
    start_date = rebalancing_dates[i]
    end_date = rebalancing_dates[i + 1]

    # 해당 기간의 데이터 추출
    period_data = stock_data.loc[start_date:end_date].pct_change().dropna(how='all', axis=1)

    # 자산의 기대 수익률 (연평균 수익률)
    annual_returns = period_data.mean() * 252
    cov_matrix = period_data.cov() * 252

    n_assets = len(annual_returns)
    w0 = np.ones(n_assets) / n_assets  # 초기 가중치 설정
    bounds = [(0, 1) for _ in range(n_assets)]  # 경계 조건: 가중치는 0 이상
    constraints = ({'type': 'eq', 'fun': lambda w: np.sum(w) - 1}) # 제약 조건: 가중치 합은  1

    # 기대 수익률과 분산-공분산 행렬 설정
    mu = annual_returns.values
    cov_matrix = cov_matrix.values

    # Maximum Sharpe Ratio Portfolio
    result_sharpe = minimize(
        max_sharpe_ratio, w0, args=(annual_returns, cov_matrix, rf),
        method='SLSQP', bounds=bounds, constraints=constraints
    )
    weights_sharpe = result_sharpe.x

    # Minimum Risk Portfolio
    result_risk = minimize(
        min_risk, w0, args=(cov_matrix,),
        method='SLSQP', bounds=bounds, constraints=constraints
    )
    weights_risk = result_risk.x

    # Maximum Return Portfolio
    result_return = minimize(
        max_return, w0, args=(annual_returns,),
        method='SLSQP', bounds=bounds, constraints=constraints
    )
    weights_return = result_return.x

    # 포트폴리오 지표 계산
    ret_sharpe, std_sharpe, sharpe_sharpe = calculate_portfolio_metrics(weights_sharpe, mu, cov_matrix, rf)
    ret_risk, std_risk, sharpe_risk = calculate_portfolio_metrics(weights_risk, mu, cov_matrix, rf)
    ret_return, std_return, sharpe_return = calculate_portfolio_metrics(weights_return, mu, cov_matrix, rf)

    # 결과 저장
    results["Maximum Sharpe Ratio"]["return"].append(ret_sharpe)
    results["Maximum Sharpe Ratio"]["std"].append(std_sharpe)
    results["Maximum Sharpe Ratio"]["sharpe"].append(sharpe_sharpe)

    results["Minimum Risk"]["return"].append(ret_risk)
    results["Minimum Risk"]["std"].append(std_risk)
    results["Minimum Risk"]["sharpe"].append(sharpe_risk)

    results["Maximum Return"]["return"].append(ret_return)
    results["Maximum Return"]["std"].append(std_return)
    results["Maximum Return"]["sharpe"].append(sharpe_return)

# 연평균 계산
mean_results = {}
for portfolio_type, metrics in results.items():
    mean_results[portfolio_type] = {
        "Return": np.mean(metrics["return"]),
        "Std": np.mean(metrics["std"]),
        "Sharpe Ratio": np.mean(metrics["sharpe"])
    }

# 결과 출력
mean_results_df = pd.DataFrame(mean_results).T
mean_results_df.columns = ["Return", "Std", "Sharpe Ratio"]

mean_results_df

,Return,Std,Sharpe Ratio
Maximum Sharpe Ratio,0.538692,0.131432,4.336740e+00
Minimum Risk,0.069285,0.046489,-1.421339e+12
Maximum Return,1.051886,0.453948,2.342425e+00
